In [2]:
using System.Threading;
using System.Threading.Tasks;
for (int i = 0; i < 1000; i++)
{
    WaitCallback waitCallback = _ =>
    {

        Thread.Sleep(1000); // to simiulate some work being done
        Console.WriteLine($"i: {i} .Thread {Thread.CurrentThread.ManagedThreadId} is running");
    };
    ThreadPool.QueueUserWorkItem(waitCallback);
}

// it will print i: 1000 1000 times when I am expecting it to print incredental numbers.
// solution: instead of use the variable i directly which is going to be reused across all of the closures, create a local variable inside the loop and use that variable inside the lambda function.

i: 1000 .Thread 7 is running
i: 1000 .Thread 24 is running
i: 1000 .Thread 25 is running
i: 1000 .Thread 6 is running
i: 1000 .Thread 22 is running
i: 1000 .Thread 15 is running
i: 1000 .Thread 27 is running
i: 1000 .Thread 26 is running
i: 1000 .Thread 4 is running
i: 1000 .Thread 12 is running
i: 1000 .Thread 23 is running
i: 1000 .Thread 28 is running
i: 1000 .Thread 7 is running
i: 1000 .Thread 25 is running
i: 1000 .Thread 24 is running
i: 1000 .Thread 22 is running
i: 1000 .Thread 6 is running
i: 1000 .Thread 27 is running
i: 1000 .Thread 15 is running
i: 1000 .Thread 12 is running
i: 1000 .Thread 4 is running
i: 1000 .Thread 26 is running
i: 1000 .Thread 23 is running
i: 1000 .Thread 28 is running
i: 1000 .Thread 7 is running
i: 1000 .Thread 25 is running
i: 1000 .Thread 22 is running
i: 1000 .Thread 26 is running
i: 1000 .Thread 15 is running
i: 1000 .Thread 6 is running
i: 1000 .Thread 27 is running
i: 1000 .Thread 24 is running
i: 1000 .Thread 4 is running
i: 1000 .Thread 12 

In [3]:
using System;
using System.Collections.Concurrent;
using System.Threading;
using System.Threading.Tasks;

for (int i = 0; i < 1000; i++)
{
    int capturedValue = i; // Create a local variable and assign the value of i to it. 
    // Every work in the queue will have its own copy of the capturedValue

    MyThreadPool.QueueUserWorkItem(delegate{
        Thread.Sleep(1000);
        Console.WriteLine($"i: {capturedValue}. Thread {Thread.CurrentThread.ManagedThreadId} is running");
    });
}

static class MyThreadPool
{
    private static readonly BlockingCollection<Action> s_workItems = new ();
    // to store things in the concurrent queue but when I want to that I want to block the thread until there is something to do.
    // the threads in the tread pool are going to try to take things from the queue to process it, if nothing is there just wait 
    public static void QueueUserWorkItem(Action action)
    {
        s_workItems.Add(action);
    }
    static MyThreadPool()
    {
        for (int i = 0; i < Environment.ProcessorCount; i++)
        {
            new Thread(()=> {}).Start();
        }
    }
}

i: 0. Thread 60 is running
i: 3. Thread 62 is running
i: 7. Thread 66 is running
i: 5. Thread 64 is running
i: 2. Thread 61 is running
i: 4. Thread 63 is running
i: 6. Thread 65 is running
i: 9. Thread 68 is running
i: 8. Thread 67 is running
i: 1. Thread 57 is running
i: 10. Thread 69 is running
i: 11. Thread 70 is running
i: 15. Thread 64 is running
i: 14. Thread 66 is running
i: 12. Thread 60 is running
i: 16. Thread 61 is running
i: 17. Thread 63 is running
i: 18. Thread 65 is running
i: 19. Thread 68 is running
i: 20. Thread 67 is running
i: 21. Thread 57 is running
i: 13. Thread 62 is running
i: 22. Thread 69 is running
i: 23. Thread 70 is running
i: 24. Thread 64 is running
i: 25. Thread 66 is running
i: 28. Thread 63 is running
i: 32. Thread 57 is running
i: 30. Thread 68 is running
i: 31. Thread 67 is running
i: 26. Thread 60 is running
i: 33. Thread 62 is running
i: 29. Thread 65 is running
i: 27. Thread 61 is running
i: 34. Thread 69 is running
i: 35. Thread 70 is running
i:

In [14]:
using System;
using System.Collections.Concurrent;
using System.Threading;
using System.Threading.Tasks;

AsyncLocal<int> capturedValue = new ();

for (int i = 0; i < 1000; i++)
{
    capturedValue.Value = i; // Create a local variable and assign the value of i to it. 
    // Every work in the queue will have its own copy of the capturedValue

    MyThreadPool.QueueUserWorkItem(delegate{
        Thread.Sleep(1000);
        Console.WriteLine($"i: {capturedValue.Value}. Thread {Thread.CurrentThread.ManagedThreadId} is running");
    });
}

static class MyThreadPool
{
    private static readonly BlockingCollection<(Action, ExecutionContext?)> s_workItems = new ();
    // to store things in the concurrent queue but when I want to that I want to block the thread until there is something to do.
    // the threads in the tread pool are going to try to take things from the queue to process it, if nothing is there just wait 
    public static void QueueUserWorkItem(Action action)
    {
        s_workItems.Add((action, ExecutionContext.Capture()));
    }
    static MyThreadPool()
    {
        for (int i = 0; i < Environment.ProcessorCount; i++)
        {
            new Thread(()=> {
              while(true){
                (Action workItem, ExecutionContext? context) = s_workItems.Take();
                if(context is null){
                  workItem();
                }else{
                  ExecutionContext.Run(context, delegate {workItem();}, null);
                  ExecutionContext.Run(context, (object? state) => ((Action)state!).Invoke(), workItem);
                }

              }
            }){IsBackground = true}.Start();
        }
    }
}

### Asynchronous
Can have asynchrony without concurrency. Asynchrony allows tasks to be paused and resumed.

<img src="https://i.gyazo.com/a4748bd9497eb9700eb9abcd0e948fb7.png" alt="Description of the image">  

Queueing up tasks to run back on the very place they were called from. It is asynchronous invocation but not necessarily concurrency.

---
### Concurrency
Can't have concurrency without asynchrony.

Concurrency involves multiple tasks making progress simultaneously, which inherently requires asynchrony to manage the execution of these tasks without blocking each other.

<img src="https://i.gyazo.com/1bd5a4d7ee3865521bb5e1d6e8a7c4d8.png" alt="Description of the image">  

Line 5 and 7 are running at the same time.

